# Stable Video Diffusion XT 1.1 on Amazon SageMaker

Stability AI's [Stable Video Diffusion XT (SVT-XT) 1.1](https://medium.com/r/?url=https%3A%2F%2Fstability.ai%2Fstable-video) foundation model is a diffusion model that takes in a still image as a conditioning frame and generates a short 4 second video. The notebook walks through configuring, creating, and invoking an [Asynchronous Inference Endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html) backed by the SVT-XT foundation model on Amazon SageMaker.

This version of the Notebook passes an encoded image in the request payload used to invoke the model.

![architecture diagram](architecture/async_inference_image_in_payload.png)

## 2: Prepare the SVD-XT Model for Inference

Steps to prepare the model for inference: 1/ Download the model artifacts from Hugging Face, 2/ add the custom inference script, 3/ create an archive file from the model artifacts, and 4/ upload the archive file to Amazon S3 for deployment.

Alternately, for steps 2.2-2.4, below, if the model archive is already available from Amazon S3, see '2.2-2.4: Alternate Method if Model Already Exists in S3', below.


### 2.1: Import Packages and Set SageMaker Variables

Import all needed packages to make sure they installed correctly.

In [ ]:
import os
import json
import shutil

import boto3
from botocore.exceptions import ClientError

import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
from sagemaker.s3 import s3_path_join

from diffusers.utils import export_to_video, make_image_grid
import ffmpeg

In [ ]:
sm_session_bucket = None

sm_session = sagemaker.Session()

if sm_session_bucket is None and sm_session is not None:
    # set to default bucket if a bucket name is not given
    sm_session_bucket = sm_session.default_bucket()
try:
    sm_role = sagemaker.get_execution_role()
except ValueError:
    iam_client = boto3.client("iam")
    sm_role = iam_client.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

In [ ]:
print(f"sagemaker role arn: {sm_role}")
print(f"sagemaker bucket: {sm_session.default_bucket()}")
print(f"sagemaker session region: {sm_session.boto_region_name}")

## 3: Set Amazon SageMaker Endpoint

If the model was previously deployed to an endpoint, set the `endpoint_name` variable.


In [ ]:
endpoint_name = "<YOUR_MODEL_ENDPOINT_NAME>"

## 4: Choosing Conditioning Images and Inference Parameters

### 4.1: Method for Encoding Conditioning Image

In [ ]:
import json
import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return "data:text/plain;base64," + base64.b64encode(image_file.read()).decode("utf-8")

def generate_request_payload(movie_title, data):
    with open(f"request_payloads/{movie_title}.json", "w") as f:
        json.dump(data, f)

### 4.2 Set Movie Name and Inference Parameters

Select one of the sets of inference parameters below and run that cell, or create your own set of parameters. Each variation creates a different video.

In [ ]:
data = {
    "image": encode_image("images_scaled/rocket.png"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 42,
}
generate_request_payload("rocket_1.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/rocket.png"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 180,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 42,
}
generate_request_payload("rocket_2.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/smoke.jpg"),
    "width": 576,
    "height": 1024,
    "num_frames": 25,
    "num_inference_steps": 50,
    "min_guidance_scale": 0.5,
    "max_guidance_scale": 1.0,
    "fps": 6,
    "motion_bucket_id": 25,
    "noise_aug_strength": 0.8,
    "decode_chunk_size": 8,
    "seed": 111142,
}
generate_request_payload("smoke.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/colored_smoke.jpg"),
    "width": 576,
    "height": 1024,
    "num_frames": 25,
    "num_inference_steps": 50,
    "min_guidance_scale": 0.5,
    "max_guidance_scale": 1.0,
    "fps": 6,
    "motion_bucket_id": 25,
    "noise_aug_strength": 0.8,
    "decode_chunk_size": 8,
    "seed": 111142,
}
generate_request_payload("colored_smoke.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/beach_bike.jpg"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 1234567890,
}
generate_request_payload("beach_bike.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/beach_bike.jpg"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 123,
}
generate_request_payload("beach_bike_2.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/waterfall.jpg"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 1234567890,
}
generate_request_payload("waterfall.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/boat_ocean.jpg"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 42,
}
generate_request_payload("boat_ocean.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/red_car.jpg"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 42,
}
generate_request_payload("red_car.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/coffee_stream.jpg"),
    "width": 576,
    "height": 1024,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 42,
}
generate_request_payload("coffee_stream.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/koi.jpg"),
    "width": 1024,
    "height": 576,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 9288258982,
}
generate_request_payload("koi.mp4", data)

In [ ]:
data = {
    "image": encode_image("images_scaled/champagne2.jpg"),
    "width": 576,
    "height": 1024,
    "num_frames": 25,
    "num_inference_steps": 25,
    "min_guidance_scale": 1.0,
    "max_guidance_scale": 3.0,
    "fps": 6,
    "motion_bucket_id": 127,
    "noise_aug_strength": 0.02,
    "decode_chunk_size": 8,
    "seed": 42,
}
generate_request_payload("champagne2.mp4", data)

## 5: Upload Request Payload and Invoke Endpoint


## 5.1: Upload Request Payload and Invoke Endpoint

Upload the JSON request payload to Amazon S3 and invoke the endpoint for inference. Invocation time for a video with 25 inference steps is about 2 minutes.


In [ ]:
def upload_file(input_location):
    return sm_session.upload_data(
        input_location,
        bucket=sm_session.default_bucket(),
        key_prefix="async_inference/input",
        extra_args={"ContentType": "application/json"},
    )

In [ ]:
# select one of the previously generated request payload files
file_name = "request_payloads/red_car.json"

# ... and upload it to S3
input_s3_location = upload_file(file_name)
print(f"Request payload location: {input_s3_location}")

In [ ]:
sm_runtime_client = boto3.client("sagemaker-runtime")

response = sm_runtime_client.invoke_endpoint_async(
    EndpointName=endpoint_name,
    InputLocation=input_s3_location,
    InvocationTimeoutSeconds=3600,
)
print(f"Model response payload location: {response['OutputLocation']}")

### 5.2: Poll for Model Response

Poll the Amazon S3 bucket for a response from the model invocation.


In [ ]:
import urllib
import time


# function reference: https://github.com/aws/amazon-sagemaker-examples/blob/main/async-inference/Async-Inference-Walkthrough-SageMaker-Python-SDK.ipynb
def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sm_session.read_s3_file(bucket=bucket, key_prefix=key)
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("Waiting for model output...")
                time.sleep(15)
                continue
            raise

In [ ]:
%%time

output = get_output(response["OutputLocation"])
print(f"Sample of output: {output[:500]}")

## 6: Frames to MP4 Video

### 6.1: Frames to MP4 Video

Convert binary objects in response to JPEGs of each frame, then combined into MP4 using Hugging Face's `diffusers.utils.export_to_video` method.


In [ ]:
import base64
from PIL import Image
from diffusers.utils import export_to_video, make_image_grid


def load_video_frames(video_frames):
    loaded_video_frames = []

    for idx, video_frame in enumerate(video_frames):
        frame = bytes(video_frame, "raw_unicode_escape")
        frame_name = f"frames_out/frame_{idx+1:02}.jpg"
        with open(frame_name, "wb") as fh:
            fh.write(base64.decodebytes(frame))

        image = Image.open(frame_name, mode="r")
        loaded_video_frames.append(image)

    return loaded_video_frames

In [ ]:
output = get_output(response["OutputLocation"])
data = json.loads(output)
frames = data["frames"]
movie_title = data["movie_title"]

loaded_video_frames = load_video_frames()

export_to_video(loaded_video_frames, f"video_out/{movie_title}", fps=6)
print(f"Video created: {movie_title}")

### 6.2: Display Frames as Grid

Display the 25 frames as a 5x5 grid using Hugging Face's `diffusers.utils.make_image_grid` method.


In [ ]:
from diffusers.utils import make_image_grid

image = make_image_grid(loaded_video_frames, 5, 5)
(width, height) = (image.width // 2, image.height // 2)
im_resized = image.resize((width, height))
display(im_resized)
im_resized.save("frames.png")

### 6.3: Display Video

Convert video CODEC to H.264 and display in notebook at 50% of actual size.


In [ ]:
import ffmpeg

output_options = {
    "crf": 20,
    "preset": "slower",
    "movflags": "faststart",
    "pix_fmt": "yuv420p",
    "vcodec": "libx264",
}


ffmpeg.input(f"video_out/{movie_title}")
    .output("video_out/tmp.mp4", **output_options)
    .run(overwrite_output=True, quiet=True)


In [ ]:
from IPython.display import Video

Video(
    url="video_out/tmp.mp4",
    width=(loaded_video_frames[0].width // 2),
    html_attributes="controls muted autoplay loop",
)

## 7: Generating of Multiple Video Variations

Generating multiple videos variations by combining the above code in a loop. In this example we are creating five variations, changing the seed each time.


In [ ]:
import random
import json

sm_runtime_client = boto3.client("sagemaker-runtime")

for i in range(3):
    seed = random.randrange(1, 9999999999)
    data = {
        "image": encode_image("images_scaled/red_car.jpg"),
        "width": 1024,
        "height": 576,
        "num_frames": 25,
        "num_inference_steps": 25,
        "min_guidance_scale": 1.0,
        "max_guidance_scale": 3.0,
        "fps": 6,
        "motion_bucket_id": 127,
        "noise_aug_strength": 0.02,
        "decode_chunk_size": 8,
        "seed": seed,
    }
    filename = f"red_car_{seed}.mp4"
    generate_request_payload(filename, data)
    input_s3_location = upload_file(file_name)

    response = sm_runtime_client.invoke_endpoint_async(
        EndpointName=endpoint_name,
        InputLocation=input_s3_location,
        InvocationTimeoutSeconds=3600,
    )

    output = get_output(response["OutputLocation"])
    data = json.loads(output)
    loaded_video_frames = load_video_frames(data["frames"])

    export_to_video(loaded_video_frames, f"video_out/{movie_title}", fps=6)
    print(f"Video created: {movie_title}")

## 8: Delete Amazon SageMaker Endpoint


In [ ]:
client_sm = boto3.client("sagemaker")
client_sm.delete_endpoint(EndpointName=endpoint_name)